# Figure notebook

Author: _Pim Dankloff_

Dependencies:

In [ ]:
# standard libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.feature_selection._mutual_info import mutual_info_regression
from scipy.optimize import root_scalar, minimize

# custom anlysis scripts
from analysis.models import szyszkowski_model, dynamic_st_model, szyszkowski
from analysis.utils import fit_model, calculate_st_at_cmc, calculate_C20
from utils.data_processing import smooth
from analysis.mixtures import fit_beta, mix_ideal, mix_non_ideal
from analysis.active_learning import ActiveLearner

## Figure 1

### illustration isotherm

import data & fit model

In [ ]:
df = pd.read_csv("data/example_experimental_isotherm/SDS.csv")
c = df["Concentration"] / 1000
st = df["Surface Tension"] / 1000
obs = (c, st)
parameters = ["cmc", "gamma_max", "Kad"]

# fit isotherm model
post_pred_1, x_new_1 = fit_model(
    obs, model=szyszkowski_model, parameters=parameters, outlier_check=False
)

# unit conversion
x_new_1 = x_new_1 * 1000
x_obs_1, y_obs_1 = obs
x_obs_1 = x_obs_1 * 1000
y_obs_1 = y_obs_1 * 1000

# mean and std of posterior predictive distribution
st_fit_mu = post_pred_1["obs"].mean(axis=0) * 1000
st_fit_std_1 = post_pred_1["obs"].std(axis=0) * 1000

plot isotherm

In [ ]:
fig, ax1 = plt.subplots(figsize=(3, 3))
ax1.plot(x_new_1, st_fit_mu, c="black", alpha=0.3, linewidth=2)
ax1.scatter(
    x_obs_1,
    y_obs_1,
    s=25,
    zorder=100,
    color="black",
    marker="x",
    alpha=1,
)


# axes settings
fontsize_labels = 10

ax1.set_ylim(35, 75)
ax1.set_yticks(np.arange(30, 76, 10))
ax1.set_xlim(0.1, 30)
ax1.set_ylabel("Surface tension / $\mathrm{mN \ m^{-1}}$", fontsize=fontsize_labels)
ax1.set_xlabel("Concentration / mM", fontsize=fontsize_labels)
ax1.set_xscale("log")
sns.despine()

# save
fig.savefig(
    "figures/figure1/example_isotherm.svg",
    dpi=300,
    bbox_inches="tight",

)

## Figure 2

### Dynamic surface tension example

Import data & fit model

In [ ]:
# import data
df = pd.read_csv("data/example_dynamic_surface_tension/C12E4_46mM.csv")
t = df["Time (s)"].values
st = df["Surface Tension (mN/m)"].values

# we slice the data to remove no (hanging) droplet region
idx = np.argmax(st) + 1
t = t[idx:]
st = st[idx:]

# # # fit kinetic model
# obs = (t, st)
# parameters = ["st_messo", "t_star", "n"]
# post_pred_1, x_new_1 = fit_model(
#     obs, model=dynamic_st_model, parameters=parameters, outlier_check=False
# )

Plot dynamic surface tension

In [ ]:
# figure
fig, ax = plt.subplots(figsize=(3, 3))
sns.despine()

# plot data and fit
ax.scatter(t, st, label="data", s=5, color="black", alpha=0.4)
# ax.plot(x_new_1, post_pred_1["obs"].mean(axis=0), label="fit", color="black", linewidth=1)

# labels
fontsize_labels = 12
ax.set_xlabel("Time / s", fontsize=fontsize_labels)
ax.set_ylabel("Surface tension / $\mathrm{mN \ m^{-1}}$", fontsize=fontsize_labels)

# save
fig.savefig(
    "figures/figure2/dynamic_surface_tension.svg", dpi=300, bbox_inches="tight"
)

## Figure 3

### Active learning illustration

Import data, fit model and calculate mutual information

In [ ]:
df = pd.read_csv("data/example_experimental_isotherm/SDS.csv")
c = df["Concentration"] / 1000
st = df["Surface Tension"] / 1000
obs = (c, st)
parameters = ["cmc", "gamma_max", "Kad"]

# fit isotherm model
post_pred_1, x_new_1 = fit_model(
    obs, model=szyszkowski_model, parameters=parameters, outlier_check=False
)

# unit conversion
x_new_1 = x_new_1 * 1000
x_obs_1, y_obs_1 = obs
x_obs_1 = x_obs_1 * 1000
y_obs_1 = y_obs_1 * 1000

# mean and std of posterior predictive distribution
st_fit_mu = post_pred_1["obs"].mean(axis=0) * 1000
st_fit_std_1 = post_pred_1["obs"].std(axis=0) * 1000

# extract mutual information
mutual_info = {}
U_total = np.zeros(post_pred_1["obs"].shape[1])
for parameter in parameters:
    U = smooth(
        mutual_info_regression(post_pred_1["obs"], post_pred_1[parameter]),
        window_size=30,
    )
    U_total += U
    mutual_info[parameter] = U

mutual_info["total"] = U_total

Plot experimental isotherm and mutual information relations

In [ ]:
fig, (ax1, ax1) = plt.subplots(2, 1, figsize=(5, 5), sharex=True)
ax1.plot(x_new_1, st_fit_mu, c="black", alpha=0.3, linewidth=2)
ax1.scatter(
    x_obs_1,
    y_obs_1,
    s=25,
    zorder=100,
    color="black",
    marker="o",
    alpha=1,
)

# plot mutual information
linewidth_mi = 2 
label_convert_dict = {
    "cmc": "CMC",
    "gamma_max": r"$\Gamma_{\max}$",
    "Kad": r"K$_{\mathrm{L}}$",
}
for parameter in parameters:
    ax1.plot(
        x_new_1,
        mutual_info[parameter],
        label=label_convert_dict[parameter],
        zorder=10,
        alpha=0.5,
        linewidth=linewidth_mi,
    )
ax1.plot(x_new_1, U_total, label="total", alpha=0.5, linewidth=linewidth_mi)


# axes settings
fontsize_labels = 10

ax1.set_ylim(35, 80)
ax1.set_xlim(0.1, 30)
ax1.set_xlim(0.1, 30)
ax1.set_xscale("log")
ax1.set_xlabel("Concentration / mM", fontsize=fontsize_labels)
ax1.set_ylabel(r"Mutual information", fontsize=fontsize_labels)
ax1.set_ylabel("Surface tension / $\mathrm{mN \ m^{-1}}$", fontsize=fontsize_labels)
ax1.legend(fontsize=10, loc="upper left", frameon=False)

sns.despine()

# save
fig.savefig(
    "figures/figure3/active_learning.svg",
    dpi=300,
    bbox_inches="tight",
)

### Model uncertainty on fitting parameters per round of AL

In [ ]:
# comment out logger to make it work #TODO logger as argument?
active_learner = ActiveLearner(model=szyszkowski_model, parameters=["cmc", "gamma_max", "Kad"])

# parameters
al_cycles = 3
cmc = 8 / 1000
gamma_max = 3.2e-5
Kad = 100
noise_level = 1
theta_true = (cmc, gamma_max, Kad)
concentrations = np.array([10, 0.1])
fit_concentrations = np.linspace(0.01, np.max(concentrations), 1000)


def sample_szyszkowski(c, theta_true, noise_level):
    c = c / 1000
    noise = np.random.normal(loc=0, scale=noise_level)
    return szyszkowski(cS0=c, theta=theta_true)*1000



st = np.array([])
fit_st = []
for c in concentrations:
    st = np.append(st, sample_szyszkowski(c=c, theta_true=theta_true, noise_level=noise_level))

for c in fit_concentrations:
    c = c / 1000
    fit_st.append(szyszkowski(cS0=c, theta=theta_true)*1000)

cmc_errors = []
gamma_max_errors = []
kad_errors = []
for i in range(al_cycles):
    x_obs = concentrations / 1000
    y_obs = np.array(st) / 1000
    obs = (x_obs, y_obs)
    c_sugg, st_sugg, post_pred = active_learner.suggest_simple(obs=obs)
    cmc_error = post_pred["cmc"].std(axis=0) / post_pred["cmc"].mean(axis=0) * 100
    gamma_max_error = post_pred["gamma_max"].std(axis=0) / post_pred["gamma_max"].mean(axis=0) * 100
    Kad_error = post_pred["Kad"].std(axis=0) / post_pred["Kad"].mean(axis=0) * 100

    cmc_errors.append(cmc_error)
    gamma_max_errors.append(
        gamma_max_error
    )
    kad_errors.append(Kad_error)

    concentrations = np.append(concentrations, c_sugg)
    st = np.append(st, sample_szyszkowski(c=c_sugg, theta_true=theta_true, noise_level=noise_level))
    print(f"Errors - CMC: {cmc_error:.2f}%, Gamma_max: {gamma_max_error:.2f}%, Kad: {Kad_error:.2f}%")
    print(f"AL cycle: {i+1}")

# last exploit point
c_sugg, st_sugg, post_pred = active_learner.suggest_simple(obs=obs)
cmc_errors.append(post_pred["cmc"].std(axis=0) / post_pred["cmc"].mean(axis=0) * 100)
gamma_max_errors.append(
    post_pred["gamma_max"].std(axis=0) / post_pred["gamma_max"].mean(axis=0) * 100
)
kad_errors.append(post_pred["Kad"].std(axis=0) / post_pred["Kad"].mean(axis=0) * 100)


plot

In [ ]:
fig, ax1 = plt.subplots(figsize=(3,3))


al_cycles_list = list(range(al_cycles+1))
ax1.plot(al_cycles_list, cmc_errors, marker='o')
ax1.plot(al_cycles_list, gamma_max_errors, marker='o')
ax1.plot(al_cycles_list, kad_errors, marker="o")

ax1.scatter(al_cycles_list, cmc_errors, marker="o", label="CMC")
ax1.scatter(al_cycles_list, gamma_max_errors, marker="o", label=r"$\Gamma_{\mathrm{max}}$")
ax1.scatter(al_cycles_list, kad_errors, marker="o", label=r"K$_\mathrm{L}$")

fontsize_labels = 12
plt.xlabel("Active learning cycle", fontsize = fontsize_labels)
plt.ylabel("Model uncertainty / %", fontsize = fontsize_labels)

# plt.ylim(0, 8)

ax1.legend(loc="center left", bbox_to_anchor=(0.55, 0.8), frameon=False, fontsize=12, handletextpad=0.1)
sns.despine()

plt.tight_layout()

plt.savefig(
    "figures/figure3/model_uncertainty.svg",
    dpi=300,
    bbox_inches="tight",
)

## Figure 4

import data and fit model

In [ ]:
# List of (experiment_tag, solution) pairs
isotherms_ids = [
    ("AULAB_PSP0076/part_3", "SDBS_1_1"),
    ("AULAB_PSP0073", "CTAB"),
    ("AULAB_PSP0076/part_7", "C12E3_1_3"),
    ("AULAB_PSP0076/part_4", "SDS_2_1"),
    ("AULAB_PSP0076/part_7", "16BAC_1_2"),
    ("AULAB_PSP0076/part_6", "C12E4_1_1"),
]

parameters = ["cmc", "gamma_max", "Kad"]

fit_results = []
for experiment_tag, solution in isotherms_ids:
    results = pd.read_csv(f"data/experiments/{experiment_tag}/results.csv")
    results_solution = results[results["solution"] == solution]
    point_types = results_solution["point type"]

    c = results_solution["concentration"] / 1000
    st = results_solution["surface tension eq. (mN/m)"] / 1000
    obs = (c, st)
    print(f"Fitting model for {solution} in {experiment_tag}...")
    post_pred, x_new = fit_model(
        obs, model=szyszkowski_model, parameters=parameters, outlier_check=False
    )
    x_new = x_new * 1000
    x_obs, y_obs = obs
    x_obs = x_obs * 1000
    y_obs = y_obs * 1000
    st_fit_mean = post_pred["obs"].mean(axis=0) * 1000
    st_fit_std = post_pred["obs"].std(axis=0) * 1000

    fit_results.append(
        {
            "experiment_tag": experiment_tag,
            "solution": solution,
            "point_types": point_types,
            "x_new": x_new,
            "x_obs": x_obs,
            "y_obs": y_obs,
            "st_fit_mean": st_fit_mean,
            "st_fit_std": st_fit_std,
            "post_pred": post_pred,
        }
    )

plot the isotherms

In [ ]:
# Suppose fit_results is a list of dicts as in the previous answer
n_isotherms = len(fit_results)
ncols = 3
nrows = int(np.ceil(n_isotherms / ncols))
fig, axes = plt.subplots(
    nrows, ncols, figsize=(2.5 * ncols, 2.5 * nrows), squeeze=False
)

fontsize_labels = 10
axes = axes.flatten()  # Flatten for easy iteration

titles = ["SDBS", "CTAB", "C$_{12}$E$_{3}$", "SDS", "16-BAC", "C$_{12}$E$_{4}$"]

for i, (ax, fit) in enumerate(zip(axes, fit_results)):
    point_types = fit["point_types"]
    explore_mask = point_types == "explore"
    exploit_mask = point_types == "exploit"
    x_new = fit["x_new"]
    st_fit_mean = fit["st_fit_mean"]
    x_obs = fit["x_obs"]
    y_obs = fit["y_obs"]

    # Plot fit
    ax.plot(
        x_new,
        st_fit_mean,
        color="black",
        alpha=0.3,
        linewidth=2,
        label="fit",
    )
    # Plot explore points
    ax.scatter(
        x_obs[explore_mask],
        y_obs[explore_mask],
        s=25,
        zorder=100,
        color="black",
        marker="o",
        alpha=1,
        label="explore" if i == 0 else None,
    )
    # Plot exploit points
    ax.scatter(
        x_obs[exploit_mask],
        y_obs[exploit_mask],
        s=25,
        zorder=100,
        color="black",
        marker="x",
        alpha=1,
        label="exploit" if i == 0 else None,
    )

    ax.set_xlabel("Concentration / mM", fontsize=fontsize_labels)
    ax.set_ylabel("Surface tension / $\mathrm{mN \ m^{-1}}$", fontsize=fontsize_labels)
    ax.set_ylim(25, 75)
    ax.set_xlim(0.001, 15)
    ax.set_xscale("log")
    if "solution" in fit:
        title = titles[i]
        ax.set_title(title, fontsize=fontsize_labels)

# Hide unused subplots
for j in range(i + 1, nrows * ncols):
    fig.delaxes(axes[j])

sns.despine()
plt.tight_layout()
fig.savefig(
    "figures/figure4/isotherm_examples.svg",
    dpi=300,
    bbox_inches="tight",
)

## Figure 5 & SI

### mixture space exploration
select mixture in the first rule for the particular mixture

import data & fit model

In [ ]:
# input
mixture = "SOS_SDS"
folder = "figureS4"

results = pd.read_csv(f"data/mixture/{mixture}/mixture_data_{mixture}.csv")
alphas = results["alpha"].unique()
parameters = ["cmc", "gamma_max", "Kad"]
fit_results = {}
for alpha in alphas:
    print(f"Fitting for alpha = {alpha}")
    results_alpha = results[results["alpha"] == alpha]
    c = results_alpha["concentration"] / 1000
    st = results_alpha["surface tension eq. (mN/m)"] / 1000
    obs = (c, st)
    post_pred, x_new = fit_model(
        obs, model=szyszkowski_model, parameters=parameters, outlier_check=False
    )
    fit_results[alpha] = {
        "post_pred": post_pred,
        "x_new": x_new,
    }
    print("done fitting for alpha =", alpha)

Extract properties

In [ ]:
data = []
alphas = results["alpha"].unique()

for alpha, fit in fit_results.items():
    row = [alpha]
    for parameter in parameters:
        if parameter == "cmc":
            row.append(fit["post_pred"][parameter].mean(axis=0) * 1000)
        else:
            row.append(fit["post_pred"][parameter].mean(axis=0))

    # Calculate st_at_cmc
    st_at_cmc = calculate_st_at_cmc(
        fit["x_new"], fit["post_pred"]
    )
    row.append(st_at_cmc * 1000)  # Convert to mN/m

    # Calculate C20
    c20 = calculate_C20(fit["x_new"], fit["post_pred"])
    row.append(c20)  # C20 in mM
    
    data.append(row)


columns = ["alpha"] + parameters + ["st_at_cmc"] + ["C20"]

df_properties = pd.DataFrame(data, columns=columns)
print(df_properties)

Plot isotherms

In [ ]:
fig, ax = plt.subplots(figsize=(5, 2.5))
size_marker = 15

# Normalize alpha values for colormap
alphas = results["alpha"].unique()
norm = plt.Normalize(alphas.min(), alphas.max())
cmap = plt.cm.coolwarm

scatter_handles = []
for alpha in alphas:
    color = cmap(norm(alpha))
    results_alpha = results[results["alpha"] == alpha]
    fit_alpha = fit_results[alpha]
    point_types = results_alpha["point type"]
    explore_mask = point_types == "explore"
    exploit_mask = point_types == "exploit"
    x_obs = results_alpha["concentration"]
    y_obs = results_alpha["surface tension eq. (mN/m)"]
    handle = ax.scatter(
        x_obs[explore_mask],
        y_obs[explore_mask],
        color=color,
        marker="o",
        zorder=15,
        s=size_marker,
        label=f"{alpha:.2f}",
    )
    scatter_handles.append(handle)
    ax.scatter(
        x_obs[exploit_mask],
        y_obs[exploit_mask],
        color=color,
        marker="x",
        s=size_marker,
        zorder=15,
    )
    x_new = fit_alpha["x_new"] * 1000
    st_fit_mean = fit_alpha["post_pred"]["obs"].mean(axis=0) * 1000  # Convert to mN/m
    ax.plot(x_new, st_fit_mean, c=color, alpha=0.5, zorder=20)

ax.set_xscale("log")
ax.set_ylim(25, 75)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

# Add legend for alpha colors
ax.legend(
    handles=scatter_handles,
    title="Molar fraction of SOS",
    # title_fontsize=6,
    title_fontproperties={"weight": "bold", "size": 6},
    fontsize=8,
    loc="lower left",
    bbox_to_anchor=(0, 0.1),
    frameon=False,
)
fontsize_labels = 10
ax.set_xlabel("Concentration / mM", fontsize=10)
ax.set_ylabel("Surface tension / $\mathrm{mN \ m^{-1}}$", fontsize=10)
sns.despine()

# save
fig.savefig(
    f"figures/{folder}/mixture_isotherm.svg",
    dpi=300,
    bbox_inches="tight",
)

interaction parameter plot: micellization 

In [ ]:
property = "C20" # options: cmc or C20

alpha_list = df_properties["alpha"].to_list()
c_mix_list = df_properties[property].to_list()

c1 = c_mix_list[-1]
c2 = c_mix_list[0]

fitted_beta = fit_beta(alpha_obs=alpha_list, c_mix_obs=c_mix_list)
alpha_fit = np.linspace(0.001, 0.999, 1000)
c_mix_ideal_list = []
c_mix_nonideal_list = []
for alpha in alpha_fit:
    c_mix_ideal = mix_ideal(c1=c1, c2=c2, alpha=alpha)
    c_mix_ideal_list.append(c_mix_ideal)
    c_mix_nonideal = mix_non_ideal(c1=c1, c2=c2, c_mix=c_mix_ideal, alpha=alpha, beta=fitted_beta)
    c_mix_nonideal_list.append(c_mix_nonideal)

# # Plotting
plt.figure(figsize=(3, 3))
plt.plot(
    alpha_fit,
    c_mix_nonideal_list,
    label="Non-Ideal",
    color="black",
    alpha=0.5,
)

plt.plot(
    alpha_fit, c_mix_ideal_list, label="Ideal", color="black", linestyle="--"
)
plt.scatter(
    alpha_list,
    c_mix_list,
    label="Experimental Data",
    zorder=30,
    color="black",
)

# plot settings
sns.despine()
fontsize_labels = 15
plt.xlabel(r"$\mathrm{\alpha}$", fontsize=fontsize_labels)
if property == "cmc":
    plt.ylabel("CMC$_{\mathrm{mix}}$ / mM", fontsize=fontsize_labels)
elif property == "C20":
    plt.ylabel("C$_{\mathrm{20}}$ / mM", fontsize=fontsize_labels)


# save
# plt.savefig(
#     f"figures/{folder}/{property}_interaction.svg",
#     dpi=300,
#     bbox_inches="tight",
# )

# Supplementary Figures

### Worthington number dependency on volume of pendant drop

In [ ]:
# import data
df = pd.read_csv("data/worthington_dropvolume/water.csv")
wo = df["Wo"]
vol = df["drop volume (uL)"]

# figure
fig, ax = plt.subplots(figsize=(3, 3))
sns.despine()

# plot experimental data + horizontal limit
ax.scatter(vol, wo, s=10, color="black", alpha=0.5)
ax.axhline(y=0.6, color="black", linestyle="--")

# labels
fontsize_labels = 12
ax.set_xlabel(r"Drop volume / $\mathrm{\mu L}$", fontsize=fontsize_labels)
ax.set_ylabel(r"Worthington number", fontsize=fontsize_labels)

# limits
ax.set_xlim(3, 7)
ax.set_ylim(-0.05, 1)

# save
# fig.savefig("figures/figure2/worthington_dropvolume.svg", dpi=figure_dpi, bbox_inches="tight")

### Comparison pendant drop measurements with literature values

In [ ]:
# import data
df = pd.read_csv("data/comparison_pendantprop_literature/data.csv")
df_water = df[df["solution"] == "water"]
df_SDS = df[df["solution"] == "SDS"]
water_mu = df_water["needle mu"].values[0]
water_std = df_water["needle std"].values[0]
SDS_mu = df_SDS["needle mu"].values[0]
SDS_std = df_SDS["needle std"].values[0]

# Experimental values
labels = ["Water", "SDS 34 mM"]
means = [water_mu, SDS_mu]
stds = [water_std, SDS_std]

# Literature values
lit_labels = ["Water (lit)", "SDS (lit)"]
lit_means = [72.2, 38]  # mN/m

# Combine labels and values for plotting
all_labels = labels + lit_labels
all_means = means + lit_means
all_stds = stds + [0, 0]  # No error bars for literature values

# Create a bar graph with error bars
plt.figure(figsize=(3, 3))

bar_width = 1  # Width of each bar
group_spacing = 1  # Spacing between groups

# Plot experimental values
x_exp = np.arange(len(labels)) * (2 * bar_width + group_spacing)
plt.bar(
    x_exp, means, yerr=stds, capsize=5, color="skyblue", alpha=0.8, label="Experimental"
)

# Plot literature values
x_lit = x_exp + bar_width
plt.bar(x_lit, lit_means, color="orange", alpha=0.8, label="Literature")

# Add labels and legend
fontsize_labels = 12
plt.xticks(
    x_exp + bar_width / 2, labels, fontsize=fontsize_labels
)  # Increased font size for x labels
plt.ylabel(
    "Surface tension / $\mathrm{mN \ m^{-1}}$", fontsize=fontsize_labels
)  # Increased font size for y-axis label
plt.legend(fontsize=8)  # Increased font size for legend
sns.despine()

# save
plt.savefig(
    "figures/figureS3/surface_tension_comparison.svg",
    dpi=300,
    bbox_inches="tight",
)

### Precision evaluation

In [ ]:
exp_tag = "AULAB_PSP0076/part_4" #SDS
df = pd.read_csv(f"data/experiments/{exp_tag}/results.csv")
df_explore = df[df["point type"] == "explore"]
df_explore_agg = (
    df_explore.groupby("concentration")["surface tension eq. (mN/m)"]
    .agg(["mean", "std"])
    .reset_index()
)

c = df_explore_agg["concentration"]
st_mu = df_explore_agg["mean"]
st_std = df_explore_agg["std"]

# # plot mean surface tension vs concentration with error bars
fig, ax = plt.subplots(figsize=(3, 3))
ax.errorbar(
    c,
    st_mu,
    yerr=2 * st_std,
    fmt="o",
    label="Mean",
    color="black",
    markersize=5,
    capsize=3,
)

# plot settings
ax.set_xscale("log")
fontsize_labels = 11
ax.set_xlabel("concentration / mM", fontsize=fontsize_labels)
plt.ylabel("Surface tension / $\mathrm{mN \ m^{-1}}$", fontsize=fontsize_labels)
# ax.set_ylim(35, 75)
sns.despine()
plt.tight_layout()

plt.savefig("figures/figureS3/precision.svg", dpi=300)

### Accuracy evaluation

In [ ]:
# pendant drop
df = pd.read_csv(f"data/eval_accuracy/A_long.csv")
c_pendant = df["Concentration"]
st_pendant = df["Surface Tension"]

# du nuoy ring
df = pd.read_excel("data/eval_accuracy/PSP0058_total_data.xlsx")
c_tensio = df["concentration (mM)"]
st_tensio = df["st tensiometer (mN/m)"]

# plot surface tension vs concentration, both methods
fig, ax = plt.subplots(figsize=(3, 3))

# pendant drop
ax.scatter(
    c_pendant,
    st_pendant,
    color="C0",
    s=50,
    label=f"Robotic Pendant Drop",
    alpha=0.5,
)

# du nouy ring
ax.scatter(
    c_tensio,
    st_tensio,
    color="C1",
    label="D$\mathrm{\ddot{u}}$ Nouy ring",
    s=50,
    marker="o",
    alpha=0.5,
)

# plot settings
ax.legend(fontsize=6)
ax.set_xscale("log")
fontsize_labels = 11
ax.set_xlabel("concentration / mM", fontsize=fontsize_labels)
plt.ylabel("Surface tension / $\mathrm{mN \ m^{-1}}$", fontsize=fontsize_labels)

ax.set_ylim(35, 75)
sns.despine()
plt.tight_layout()

plt.savefig("figures/figureS3/accuracy.svg", dpi=300)